<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 main.py




In [29]:
from pyngrok import ngrok

import os  
import shutil  

from flask_bootstrap import Bootstrap
from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000



In [ ]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/templates"
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
app = Flask(__name__, template_folder=TEMPLATE)

app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/", methods=["GET", "POST"])
def upload_file():
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            
        os.system("rembg p /content/kirli   /content/temiz")
        os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input /content/temiz --output /content/modelFolder")
        os.system("obj2gltf -i /content/modelFolder/model.obj -o /content/static/model.glb")
        return render_template("showModel.html")
    return render_template("index.html")


@app.route("/denemeeee")
def Ac():
    return render_template("dene.html")


if __name__ == "__main__":
    app.run()